Firstly, we import the libraries that we're going to use in this part.

In [6]:
import pytesseract
from pytesseract import Output
import re
import cv2
import numpy as np
from gtts import gTTS 
from IPython.display import Audio 
import time
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract'

Now we define the function that will take the important information of the text (name of the shop, date of the ticket, address and price)

In [7]:
def text_parsing(path):
    
    img = cv2.imread(path)

    st = time.time()
    text = pytesseract.image_to_data(img, output_type=Output.DICT)
    ft = time.time()
    print("Total Time: " + str(ft-st))

    date_exp1 = '^(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](19|20)\d\d$'
    date_exp2 = '\d\d[- /.]\d\d\d\d\d\d\d'
    date_exp3 = '\d\d\d\d\d[- /.]\d\d\d\d'
    address_exp = 'NO[.]'
    price_exp = '(\d+\.\d{1,2})'
    price_list = []
    price = 0.00
    date = ''
    address = ''
    name = ''
    name_start = True

    n_boxes = len(text['text'])
    for i in range(n_boxes):
        if (text['conf'][i] > 60):
            if name_start and re.match('.*', text['text'][i]) and text['text'][i] != " ": 
                name_start = False
                aux = i + 1
                (x, y, w, h) = (text['left'][i], text['top'][i], text['width'][i], text['height'][i])
                name = text['text'][i]
                while not re.match('$', text['text'][aux]):
                    w = w + text['width'][aux] + 10 
                    name = name + " " + text['text'][aux]
                    aux += 1
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            if re.match(date_exp1, text['text'][i]):
                (x, y, w, h) = (text['left'][i], text['top'][i], text['width'][i], text['height'][i])
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                date = text['text'][i]
            elif re.match(date_exp2, text['text'][i]):
                (x, y, w, h) = (text['left'][i], text['top'][i], text['width'][i], text['height'][i])
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                date = text['text'][i]
            elif re.match(date_exp3, text['text'][i]):
                (x, y, w, h) = (text['left'][i], text['top'][i], text['width'][i], text['height'][i])
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                date = text['text'][i]
            if re.match(address_exp, text['text'][i]):
                aux = i+1
                (x, y, w, h) = (text['left'][i], text['top'][i], text['width'][i], text['height'][i])
                address = text['text'][i]
                while not re.match("$", text['text'][aux]):
                    w = w + text['width'][aux] + 10 
                    address = address + " " + text['text'][aux]
                    aux += 1
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            if re.match(price_exp, text['text'][i]):
                (x, y, w, h) = (text['left'][i], text['top'][i], text['width'][i], text['height'][i])
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                price_list.append(float(text['text'][i]))    
    try:
        price_list = [float(i) for i in price_list]
        price_list = set(price_list)
        if len(price_list) >= 2:
            price_list.remove(max(price_list))
        if len(price_list) != 0:
            price = max(price_list)
    except:
        price = 0.00

    cv2.imshow('img', img)
    cv2.waitKey(0)

    return name, date, address, price

This function formats the text to be put into an audio:

In [8]:
def format_text(name, date, address, price):
    text = "The ticket is from " + name + ", located in " + address + ". It's from " + date + "and the total price is " + str(price)
    return text

We call the functions that get, parse and format the text.

In [9]:
path = "C:\\Users\\Usuari\\Desktop\\Database\\Tickets\\imges\\test\\470.jpg"

name, date, address, price = text_parsing(path)

final_text = format_text(name, date, address, price)


Total Time: 6.746957302093506


Finally, we call the functions that create the audio file:

In [10]:
tts = gTTS(final_text) 
tts.save('final_text.mp3')

sound_file = 'final_text.mp3'
Audio(sound_file, autoplay=True) 